<a href="https://colab.research.google.com/github/mrefugionv/S14---APLICACION-LIBROS-SQL/blob/main/P14_SQL_LIBROS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROYECTO SQL
María del Refugio Noriega Vargas

## Descripción del proyecto

Te han dado una base de datos de uno de los servicios de aplicaciones para los amantes de los libros. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. Esta información se utilizará para generar una propuesta de valor para un nuevo producto.

# Objetivos del estudio

* Encuentra el número de libros publicados después del 1 de enero de 2000.
* Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.
* Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).
* Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.
* Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

## Estudio de tablas

### Conexión a base de datos

In [ ]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
db_config = {
 'user': 'practicum_student', # username
 'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7', # password
 'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
 'port': 5432, # connection port
 'db': 'data-analyst-final-project-db' # the name of the database
 }
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# La conexión se almacena en la variable engine .
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


### Tablas

#### Books

Contiene datos sobre libros:

- `book_id` — identificación del libro
- `author_id` — identificación del autor o autora
- `title` — título
- `num_pages` — número de páginas
- `publication_date` — fecha de la publicación
- `publisher_id` — identificación de la editorial

In [ ]:
query_1 = """
SELECT *
FROM books
Limit 5;
"""
pd.io.sql.read_sql(query_1, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


#### Authors

Contiene datos sobre autores:
- `author_id` — identificación del autor o autora
- `author` — el autor o la autora

In [ ]:
query_2 = """
SELECT *
FROM authors
LIMIT 5;
"""
pd.io.sql.read_sql(query_2, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


#### Publishers

Contiene datos sobre editoriales:
- `publisher_id` — identificación de la editorial
- `publisher` — la editorial

In [ ]:
query_3 = """
SELECT *
FROM publishers
LIMIT 5;
"""
pd.io.sql.read_sql(query_3, con=engine )

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


#### Ratings

Contiene datos sobre las calificaciones de usuarios:
- `rating_id` — identificación de la calificación
- `book_id` — identificación del libro
- `username` — el nombre del usuario que revisó el libro
- `rating` — calificación

In [ ]:
query_4 = """
SELECT *
FROM ratings
LIMIT 5;
"""
pd.io.sql.read_sql(query_4,con=engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


#### Reviews

Contiene datos sobre las reseñas de los y las clientes:
- `review_id` — identificación de la reseña
- `book_id` — identificación del libro
- `username` — el nombre del usuario que revisó el libro
- `text` — el texto de la reseña


In [ ]:
query_5 = """
SELECT *
FROM reviews
LIMIT 5;
"""
pd.io.sql.read_sql(query_5,con=engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Diagrama de datos

<img src=https://practicum-content.s3.us-west-1.amazonaws.com/resources/moved_Untitled_-_2020-07-02T142019.920_1593688954.png>




##  Consultas

### Número de libros publicados después del 1 de enero de 2000.

En la base de datos hay 1000 libros, de los cuales 819 fueron publicados antes del 1 de enero del 2000.

In [ ]:
query_6 = """
SELECT COUNT(book_id) AS book
FROM books
WHERE publication_date > '2000-01-01';
"""

pd.io.sql.read_sql(query_6, con=engine)

,book
0,819


###  Número de reseñas de usuarios y la calificación promedio para cada libro.

Tenemos registros de 1000 libros.

Observamos que el libro con 5 estrellas de rating que más reseñas tiene es el 17, con 16 reseñas. Y el libro con el ranking más bajo (1.5 estrellas) solo tiene 4 reseñas, es el libro con id 303.

In [ ]:
query_7 = """
SELECT  ratings.book_id,
COUNT(reviews.review_id) AS number_reviews,
AVG(ratings.rating) AS mean_rate
FROM reviews
FULL JOIN  ratings ON reviews.book_id = ratings.book_id
GROUP BY ratings.book_id
ORDER BY mean_rate DESC , number_reviews DESC;
"""
pd.io.sql.read_sql(query_7,con=engine)

,book_id,number_reviews,mean_rate
0,17,16,5.00
1,553,12,5.00
2,444,9,5.00
3,347,6,5.00
4,610,4,5.00
...,...,...,...
995,915,12,2.25
996,202,9,2.00
997,371,4,2.00
998,316,4,2.00


###  Editorial que ha publicado el mayor número de libros con más de 50 páginas

La editorial **Penguin Books**, con id 212, es la que ha publicado mayor número de libros de más de 50 páginas; ha publicado 42. Le sigue la editorial Vintage con 31 libros de más de 50 páginas.

In [ ]:
query_8 = """
SELECT books.publisher_id,
publishers.publisher,
COUNT(book_id) AS number_books
FROM books
FULL JOIN publishers ON publishers.publisher_id = books.publisher_id
WHERE books.num_pages > 50
GROUP BY books.publisher_id,publishers.publisher
ORDER BY number_books DESC
LIMIT 3;
"""

pd.io.sql.read_sql(query_8,con=engine)

,publisher_id,publisher,number_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25


### Autor que tiene la más alta calificación promedio del libro (solo libros con al menos 50 calificaciones).

**J.K. Rowling/Mary GrandPré** con el libro "Harry Potter and the Prisoner of Azkaban" (id = 302), ha acumulado una calificación promedio de **4.41**  con una cantidad de 82 evaluaciones.

In [ ]:
query_9 = """
SELECT AVG(ratings.rating) AS mean_rate,
COUNT(ratings.rating_id) AS rates_count,
books.book_id,
books.title,
authors.author
FROM ratings
RIGHT JOIN books ON books.book_id = ratings.book_id
RIGHT JOIN authors ON authors.author_id = books.author_id
GROUP BY ratings.book_id, books.book_id,authors.author
HAVING COUNT(ratings.rating_id) > 50
ORDER BY mean_rate DESC
LIMIT 3;
"""
pd.io.sql.read_sql(query_9,con=engine)

,mean_rate,rates_count,book_id,title,author
0,4.414634,82,302,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré
1,4.391892,74,722,The Fellowship of the Ring (The Lord of the Ri...,J.R.R. Tolkien
2,4.287500,80,299,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling/Mary GrandPré


###  Número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

Los usuarios que califiacron más de 50 libros diferentes, en promedio escriben 1,394.53 reseñas escritas.


In [ ]:
query_10 = """
SELECT  AVG(reviews.review_id) AS avg_text_reviews
FROM ratings
LEFT JOIN reviews ON ratings.username = reviews.username
HAVING COUNT(DISTINCT(ratings.book_id)) > 50 ;
"""

pd.io.sql.read_sql(query_10, con= engine)

,avg_text_reviews
0,1394.537916
